In [1]:
import ROOT
from ROOT import gStyle
import numpy as np

import ctypes

def biner(o_bin,m_bin,f_bin,f_step,l_step):
    a=np.arange(o_bin,m_bin,f_step)
    b=np.arange(m_bin,f_bin+l_step,l_step)
    return np.concatenate((a,b))

def normalization(hist_list):
    for hist in hist_list:
        for i in range(0,len(rebining)):
            value=hist.GetBinContent(i)
            error=hist.GetBinError(i)
            if i<(len(rebining)-1):
                if hist==signal:
                    sf=1*(rebining[i]-rebining[i-1])
                else:  
                    sf=(rebining[i]-rebining[i-1])    
            hist.SetBinContent(i,value/sf)
            hist.SetBinError(i,error/sf)

Welcome to JupyROOT 6.22/00


In [2]:
histos_no_rebin={"lep1_phi_topo":[],
                 "lep1_phi_cuts":[],
                 "lep2_phi_topo":[],
                 "lep2_phi_cuts":[],
                 "jet_n_topo":[],
                 "jet_n_topo_dphi_btag_iso_pt1_pt2_mass":[],
                 "delta_phi":[],
                 "delta_phi_topo":[],
                 "delta_phi_cuts_butphi":[],
                 "ratio_ptjet_zpt_cuts":[],
                 "ratio_lpt_tpt_cuts":[],
      }
histos_rebin={"lep1_pt_topo":[180,1,10],
              "lep1_pt_topo_dphi_btag_iso_pt2_mass":[180,1,10],
              "lep2_pt_topo":[180,1,10],
              "lep2_pt_topo_dphi_btag_iso_pt1_mass":[180,1,10],
              "sumlep_pt_topo":[180,1,10],
              "sumlep_pt_topo_dphi_btag_iso_pt1_pt2_mass":[180,1,10],
              "MET_topo":[120,1,5],
              "MET_topo_dphi_btag_iso_pt1_pt2_mass":[120,1,5],
             "inv_mass_topo":[120,1,5],
              "h_inv_mass_topo_dphi_btag_iso_pt1_pt2":[120,1,10],
              "Z_pt_topo":[180,5,10],
              "Z_pt_cuts":[180,5,10],}
histos={}
histos.update(histos_no_rebin)
histos.update(histos_rebin)
    
    
for i in histos:
    print(i)
    MC=ROOT.TFile.Open("MC.root","READ")
    Data = ROOT.TFile.Open("Data.root","READ")
    Signal=ROOT.TFile.Open("Zee.root","READ")
    VV=ROOT.TFile.Open("VV.root","READ")
    Wjets=ROOT.TFile.Open("Wjets.root","READ")
    Zjets=ROOT.TFile.Open("Ztautau_Sherpa.root","READ")
    Ttbar=ROOT.TFile.Open("ttbar.root","READ")
    SingleTop=ROOT.TFile.Open("singletop.root","READ")

    gStyle.SetOptStat(1111111)
    a=i
    
    mc=MC.Get(a)
    data=Data.Get(a)
    signal=Signal.Get(a)
    vv=VV.Get(a)
    w=Wjets.Get(a)
    z=Zjets.Get(a)
    tt=Ttbar.Get(a)
    t=SingleTop.Get(a)
    
    mc.SetDirectory(0)
    signal.SetDirectory(0)
    data.SetDirectory(0)
    vv.SetDirectory(0)
    w.SetDirectory(0)
    z.SetDirectory(0)
    tt.SetDirectory(0)
    t.SetDirectory(0)
    
    MC.Close()
    Signal.Close()
    Data.Close()
    VV.Close()
    Wjets.Close()
    Zjets.Close()
    Ttbar.Close()
    SingleTop.Close()

    hs = ROOT.THStack("hs","")
    
    if i in histos_rebin:
        ob=data.GetNbinsX()
        s=data.GetXaxis().GetBinLowEdge(1)
        e=data.GetXaxis().GetBinUpEdge(data.GetNbinsX())
        rebining=biner(s,histos[i][0],e,histos[i][1],histos[i][2])
        nb=len(rebining)-1
        mc=mc.Rebin(nb,"mc",rebining)
        signal=signal.Rebin(nb,"signal",rebining)
        data=data.Rebin(nb,"data",rebining)
        vv=vv.Rebin(nb,"vv",rebining)
        tt=tt.Rebin(nb,"tt",rebining)
        t=t.Rebin(nb,"t",rebining)
        z=z.Rebin(nb,"z",rebining)
        w=w.Rebin(nb,"w",rebining)
        
        if histos[i][1]!=histos[i][2]:
            hist_list=[signal,data,vv,tt,t,z,w,mc]
            normalization(hist_list)

    w.SetFillColor(ROOT.kGreen)
    w.SetLineColor(ROOT.kGreen)
    z.SetFillColor(ROOT.kRed)
    z.SetLineColor(ROOT.kRed)
    vv.SetFillColor(ROOT.kBlue)
    vv.SetLineColor(ROOT.kBlue)
    tt.SetFillColor(ROOT.kYellow)
    tt.SetLineColor(ROOT.kYellow)
    t.SetFillColor(ROOT.kCyan)
    t.SetLineColor(ROOT.kCyan)
    data.SetLineColor(ROOT.kBlack)



    hs.Add(vv)
    hs.Add(w)
    hs.Add(z)
    hs.Add(tt)
    hs.Add(t)
    hs.Add(signal)

    ratio=mc.Clone()
    ratio.Divide(data)

    gStyle.SetOptStat(1111111)

    canvas3 = ROOT.TCanvas("canvas3")
    canvas3.cd()

    pad1 = ROOT . TPad (" pad1 "," pad1 " ,0 ,0.35 ,1 ,1)

    pad1.Draw ()
    pad1.cd ()




    data.Draw("pe")
    hs.Draw("HIST same")
    #if "reco_mass" in i:
    #pad1.SetLogy()


    data . GetYaxis (). SetRangeUser (1 ,1.2*data.GetBinContent(data.GetMaximumBin()))
    data . GetXaxis (). SetRangeUser (-3,400)
    if "inv_mass" in i:
        data . GetXaxis (). SetRangeUser (50,150)

    hs . GetXaxis (). SetRangeUser (20e3 ,30e4)
    hs . GetYaxis (). SetRangeUser (0 ,40)
    legend = ROOT . TLegend (0.8 ,0.4 ,0.9 ,0.6)
    legend . AddEntry ( vv ,"Diboson")
    legend . AddEntry ( data ," Data ")
    legend . AddEntry ( signal ," Signal ")
    legend . AddEntry ( w ," W+jets ")
    legend . AddEntry ( z ," Z+jets ")
    legend . AddEntry ( t ," Single top ")
    legend . AddEntry ( tt ," ttbar ")

    legend . SetLineWidth (0)
    legend . Draw ()



    hs.SetTitle(a)

    canvas3.cd()
    pad2 = ROOT . TPad (" pad2 "," pad2 " ,0 ,0.05 ,1 ,0.25)
    pad2.Draw ()
    pad2.cd ()
    ratio.Draw ("pe")
    ratio.SetStats(0)
    ratio . GetYaxis (). SetRangeUser (0.5 ,1.5)
    ratio . GetXaxis (). SetRangeUser (0 ,300e3)
    if "inv_mass" in i:
        ratio . GetXaxis (). SetRangeUser (50 ,150)
    ratio . GetYaxis (). SetTitle ("MC/DATA")
    ratio . GetYaxis (). SetTitleSize (0.15)
    ratio . GetYaxis (). SetTitleOffset (0.25)
    line = ROOT . TLine (0 ,1 ,300e3 ,1)
    line . SetLineColor ( ROOT . kBlack )
    line . SetLineWidth (2)
    line . Draw (" same ")
        







    canvas3.Print(a+".pdf")
    
    if i=="ratio_lpt_tpt_cuts":
        dic={"data":data,"signal":signal,"vv":vv,"w":w,"z":z,"tt":tt,"t":t}
        for i in dic:
            error=ctypes.c_double()
            a=dic[i].IntegralAndError(0,1000,error)
            print(i)
            print(str(a)+" +E "+str(error))




lep1_phi_topo
lep1_phi_cuts
lep2_phi_topo
lep2_phi_cuts
jet_n_topo
jet_n_topo_dphi_btag_iso_pt1_pt2_mass
delta_phi
delta_phi_topo
delta_phi_cuts_butphi
ratio_ptjet_zpt_cuts
ratio_lpt_tpt_cuts
data
1322526.0 +E c_double(1150.0113042922665)
signal
1234484.4479980469 +E c_double(578.9511007066403)
vv
17710.2540435791 +E c_double(36.51779230282724)
w
32.32278394699097 +E c_double(15.37363465431243)
z
68.79355134814978 +E c_double(9.70963435524295)
tt
1534.8517370820045 +E c_double(14.89016458584271)
t
288.4524275660515 +E c_double(6.242633895788698)
lep1_pt_topo
lep1_pt_topo_dphi_btag_iso_pt2_mass
lep2_pt_topo
lep2_pt_topo_dphi_btag_iso_pt1_mass
sumlep_pt_topo
sumlep_pt_topo_dphi_btag_iso_pt1_pt2_mass
MET_topo
MET_topo_dphi_btag_iso_pt1_pt2_mass
inv_mass_topo
h_inv_mass_topo_dphi_btag_iso_pt1_pt2
Z_pt_topo
Z_pt_cuts


Info in <TCanvas::Print>: pdf file lep1_phi_topo.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file lep1_phi_cuts.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file lep2_phi_topo.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file lep2_phi_cuts.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file jet_n_topo.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Error in <TPad::Range>: illegal world coordinates range: x1=-3.500000, y1=0.000000, x2=11.500000, y2=0.000000
Error in <TPad::RangeAxis>: illegal axis coordinates range: xmin=-2.000000, ymin=0.000000, xmax=10.000000, ymax=0.000000
Error in <TPad::Range>: illegal world coordinat

### PLOT SUM OF TWO HISTOGRAMS

In [3]:
ratio = ROOT.TH1F( 'total', 'This is the total distribution', 200, 0, 200 )
hs_mc = ROOT.TH1F("hs_mc",'This is the total distribution', 200, 0, 200)
hs_data = ROOT.TH1F("hs_data",'This is the total distribution', 200, 0, 200)
sum_histos={"lep1_pt_topo_dphi_btag_iso_pt2_mass":[120,1,10],"lep2_pt_topo_dphi_btag_iso_pt1_mass":[120,1,10]}
title='pt1+pt2'
for a in sum_histos:
    MC=ROOT.TFile.Open("MC.root","READ")
    Data = ROOT.TFile.Open("Data.root","READ")

    gStyle.SetOptStat(1111111)
    mc=MC.Get(a)
    data=Data.Get(a)

    mc.SetDirectory(0)
    data.SetDirectory(0)
    
    MC.Close()
    Data.Close()

    hs_mc.Add(mc)
    hs_data.Add(data)
    

ratio.Divide(hs_mc,hs_data)

gStyle.SetOptStat(1111111)

canvas3 = ROOT.TCanvas("canvas3")
canvas3.cd()

pad1 = ROOT . TPad (" pad1 "," pad1 " ,0 ,0.35 ,1 ,1)

pad1.Draw ()
pad1.cd ()

hs_data.Draw("pe")
hs_mc.Draw("HIST same")

hs_data . GetXaxis (). SetRangeUser (0,200)
hs_mc . GetXaxis (). SetRangeUser (0,200)

legend = ROOT . TLegend (0.8 ,0.4 ,0.9 ,0.6)
legend . AddEntry ( hs_mc ,"MC")
legend . AddEntry ( hs_data ," Data ")

legend . SetLineWidth (0)
legend . Draw ()



hs_mc.SetTitle(title)

canvas3.cd()
pad2 = ROOT . TPad (" pad2 "," pad2 " ,0 ,0.05 ,1 ,0.25)
pad2.Draw ()
pad2.cd ()
ratio.Draw ("pe")
ratio.SetStats(0)
ratio . GetYaxis (). SetRangeUser (0.5 ,1.5)
ratio . GetXaxis (). SetRangeUser (0 ,200)
ratio . GetYaxis (). SetTitle ("MC/DATA")
ratio . GetYaxis (). SetTitleSize (0.15)
ratio . GetYaxis (). SetTitleOffset (0.25)
line = ROOT . TLine (0 ,1 ,200 ,1)
line . SetLineColor ( ROOT . kBlack )
line . SetLineWidth (2)
line . Draw (" same ")
canvas3.Print(title+".pdf")

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file pt1+pt2.pdf has been created
